In [1]:
import pandas as pd

train_data = pd.read_csv('train2.csv', encoding = 'cp949')
test_data = pd.read_csv('test2.csv' , encoding = 'cp949')

In [3]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier  # RandomForest 모델

# 1. 데이터 준비 및 전처리
# 'train_data' DataFrame에서 ID, 시술 시기 코드, 시술 유형 칼럼 등을 제외한 나머지가 feature,
# '임신 성공 여부'가 target입니다.

# (1) 클래스별 데이터 분리 (다운 샘플링 전)
df_majority = train_data[train_data['임신 성공 여부'] == 0]
df_minority = train_data[train_data['임신 성공 여부'] == 1]

# (2) 다수 클래스(0)를 소수 클래스(1)의 수와 맞춰 다운 샘플링
df_majority_downsampled = resample(
    df_majority,
    replace=False,                  # 중복 없이 샘플링
    n_samples=len(df_minority),     # 소수 클래스와 동일한 수
    random_state=42                 # 재현성을 위한 random_state 설정
)

# (3) 다운 샘플링된 데이터와 소수 클래스 데이터를 결합
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# 2. 피처와 타겟 변수 분리
# 제외할 칼럼: 'ID', '시술 시기 코드', '시술 유형', 그리고 '임신 성공 여부' (타겟 변수)
cols_to_drop = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부',
                '여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
                'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수',
                '동결 배아 사용 여부', '신선 배아 사용 여부']
X = df_downsampled.drop(columns=cols_to_drop)
y = df_downsampled['임신 성공 여부']

# 3. 학습/검증 데이터 분리 (Stratify 옵션으로 클래스 비율 유지)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. 모델 학습 (RandomForest 사용)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# 5. 예측 및 평가
y_pred = clf.predict(X_val)
y_prob = clf.predict_proba(X_val)[:, 1]  # Positive class 확률

# 평가 지표 출력
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
conf_mat = confusion_matrix(y_val, y_pred)

print("Accuracy: {:.4f}".format(accuracy))
print("ROC-AUC: {:.4f}".format(roc_auc))
print("Confusion Matrix:\n", conf_mat)

Accuracy: 0.6193
ROC-AUC: 0.6704
Confusion Matrix:
 [[11205  8417]
 [ 6521 13100]]


In [7]:
# 6. 다른 threshold 값에 따른 평가 지표 출력
from sklearn.metrics import f1_score
thresholds = [0.55, 0.6, 0.65]

for thresh in thresholds:
    # 해당 threshold 이상이면 1로 분류
    y_pred_thresh = (y_prob >= thresh).astype(int)
    
    # 평가 지표 계산
    acc = accuracy_score(y_val, y_pred_thresh)
    roc_auc = roc_auc_score(y_val, y_prob)  # ROC-AUC는 확률값을 사용하므로 threshold와 무관
    conf_mat = confusion_matrix(y_val, y_pred_thresh)
    
    print(f"Threshold: {thresh}")
    print("Accuracy: {:.4f}".format(acc))
    print("ROC-AUC: {:.4f}".format(roc_auc))
    print("Confusion Matrix:\n", conf_mat)
    print("F1 score : {:.4f}".format(f1_score(y_val,y_pred)))
    print("-" * 50)

Threshold: 0.55
Accuracy: 0.6105
ROC-AUC: 0.6704
Confusion Matrix:
 [[12467  7155]
 [ 8131 11490]]
F1 score : 0.6369
--------------------------------------------------
Threshold: 0.6
Accuracy: 0.6030
ROC-AUC: 0.6704
Confusion Matrix:
 [[13513  6109]
 [ 9470 10151]]
F1 score : 0.6369
--------------------------------------------------
Threshold: 0.65
Accuracy: 0.5884
ROC-AUC: 0.6704
Confusion Matrix:
 [[14572  5050]
 [11101  8520]]
F1 score : 0.6369
--------------------------------------------------


In [15]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# 1. 데이터 준비 및 전처리
# 'train_data' DataFrame에서 ID, 시술 시기 코드, 시술 유형 칼럼 등을 제외한 나머지가 feature,
# '임신 성공 여부'가 target입니다.

# (1) 클래스별 데이터 분리 (다운 샘플링 전)
df_majority = train_data[train_data['임신 성공 여부'] == 0]
df_minority = train_data[train_data['임신 성공 여부'] == 1]

# (2) 다수 클래스(0)를 소수 클래스(1)의 수와 맞춰 다운 샘플링
df_majority_downsampled = resample(
    df_majority,
    replace=False,                  # 중복 없이 샘플링
    n_samples=len(df_minority),     # 소수 클래스와 동일한 수
    random_state=42                 # 재현성을 위한 random_state 설정
)

# (3) 다운 샘플링된 데이터와 소수 클래스 데이터를 결합
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# 2. 피처와 타겟 변수 분리
# 제외할 칼럼: 'ID', '시술 시기 코드', '시술 유형', 그리고 '임신 성공 여부' (타겟 변수)
cols_to_drop = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부',
                '여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
                'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수',
                '동결 배아 사용 여부', '신선 배아 사용 여부']
X = df_downsampled.drop(columns=cols_to_drop)
y = df_downsampled['임신 성공 여부']

# 3. 학습/검증 데이터 분리 (Stratify 옵션으로 클래스 비율 유지)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. 하이퍼파라미터 튜닝을 위한 그리드 서치 수행
# AUC ROC를 최대화하는 방향으로 탐색합니다.
param_grid = {
    'n_estimators': [300],
    'max_depth': [100],
    'min_samples_split': [10],
    'min_samples_leaf': [4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='roc_auc',  # AUC ROC 점수를 기준으로 최적 파라미터 탐색
    cv=5,
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)

print("최적의 하이퍼파라미터:", grid_search.best_params_)

# 5. 최적 모델로 예측 및 평가
best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_val)
y_prob = best_rf.predict_proba(X_val)[:, 1]  # Positive class 확률

accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
conf_mat = confusion_matrix(y_val, y_pred)

print("Accuracy: {:.4f}".format(accuracy))
print("ROC-AUC: {:.4f}".format(roc_auc))
print("Confusion Matrix:\n", conf_mat)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
최적의 하이퍼파라미터: {'max_depth': 100, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
Accuracy: 0.6598
ROC-AUC: 0.7125
Confusion Matrix:
 [[10616  9006]
 [ 4344 15277]]


In [19]:
###########3

# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# 1. 데이터 준비 및 전처리
# 'train_data' DataFrame에서 ID, 시술 시기 코드, 시술 유형 칼럼 등을 제외한 나머지가 feature,
# '임신 성공 여부'가 target입니다.

# (1) 클래스별 데이터 분리 (다운 샘플링 전)
df_majority = train_data[train_data['임신 성공 여부'] == 0]
df_minority = train_data[train_data['임신 성공 여부'] == 1]

# (2) 다수 클래스(0)를 소수 클래스(1)의 수와 맞춰 다운 샘플링
df_majority_downsampled = resample(
    df_majority,
    replace=False,                  # 중복 없이 샘플링
    n_samples=len(df_minority),     # 소수 클래스와 동일한 수
    random_state=42                 # 재현성을 위한 random_state 설정
)

# (3) 다운 샘플링된 데이터와 소수 클래스 데이터를 결합
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# 2. 피처와 타겟 변수 분리
# 제외할 칼럼: 'ID', '시술 시기 코드', '시술 유형', 그리고 '임신 성공 여부' (타겟 변수)
cols_to_drop = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부',
                '여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
                'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수',
                '동결 배아 사용 여부', '신선 배아 사용 여부']
X = df_downsampled.drop(columns=cols_to_drop)
y = df_downsampled['임신 성공 여부']

# 3. 학습/검증 데이터 분리 (Stratify 옵션으로 클래스 비율 유지)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. 하이퍼파라미터 튜닝을 위한 그리드 서치 수행
# AUC ROC를 최대화하는 방향으로 탐색합니다.
param_grid = {
    'n_estimators': [300, 500],
    'max_depth': [200, 300],
    'min_samples_split': [10, 20],
    'min_samples_leaf': [4, 8]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='roc_auc',  # AUC ROC 점수를 기준으로 최적 파라미터 탐색
    cv=5,
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)

print("최적의 하이퍼파라미터:", grid_search.best_params_)

# 5. 최적 모델로 예측 및 평가
best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_val)
y_prob = best_rf.predict_proba(X_val)[:, 1]  # Positive class 확률

accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
conf_mat = confusion_matrix(y_val, y_pred)

print("Accuracy: {:.4f}".format(accuracy))
print("ROC-AUC: {:.4f}".format(roc_auc))
print("Confusion Matrix:\n", conf_mat)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
최적의 하이퍼파라미터: {'max_depth': 200, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 500}
Accuracy: 0.6626
ROC-AUC: 0.7166
Confusion Matrix:
 [[10598  9024]
 [ 4216 15405]]


In [41]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

# 1. 데이터 준비 및 전처리
df_majority = train_data[train_data['임신 성공 여부'] == 0]
df_minority = train_data[train_data['임신 성공 여부'] == 1]

# (1) 다수 클래스(0)를 소수 클래스(1)의 수와 맞춰 다운샘플링
df_majority_downsampled = resample(
    df_majority,
    replace=False,
    n_samples=len(df_minority),
    random_state=42
)

# (2) 다운샘플링된 다수 클래스와 소수 클래스를 결합
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# (3) 피처와 타겟 변수 분리
cols_to_drop = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부',
                '여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
                'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수',
                '동결 배아 사용 여부', '신선 배아 사용 여부']
X = df_downsampled.drop(columns=cols_to_drop)
y = df_downsampled['임신 성공 여부']

# (4) 학습/검증 데이터 분리 (Stratify 옵션으로 클래스 비율 유지)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

####################################
# Model 1: RandomForest (GridSearchCV 활용)
param_grid_rf = {
    'n_estimators': [500],
    'max_depth': [200, 300],
    'min_samples_split': [20],
    'min_samples_leaf': [8]
}
rf = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    scoring='roc_auc',  # AUC ROC 기준 최적화
    cv=5,
    n_jobs=-1,
    verbose=1
)
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_
print("RandomForest 최적의 하이퍼파라미터:", grid_search_rf.best_params_)

####################################
# Model 2: LightGBM (기본 모델, GridSearchCV 활용)
# 파라미터 값들을 리스트로 감싸줍니다.
param_grid_lgb_2 = {
    'learning_rate': [0.01],
    'max_depth': [-1],
    'n_estimators': [300],
    'num_leaves': [31]
}
lgb_model_plain = lgb.LGBMClassifier(random_state=42)
grid_lgb_model_plain = GridSearchCV(
    estimator=lgb_model_plain,  # lgb_estimator가 아닌 lgb_model_plain 사용
    param_grid=param_grid_lgb_2,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1
)
grid_lgb_model_plain.fit(X_train, y_train)
best_lgb_plain = grid_lgb_model_plain.best_estimator_
print("LightGBM (기본 모델) 최적의 하이퍼파라미터:", grid_lgb_model_plain.best_params_)

####################################
# Model 3: LightGBM (GridSearchCV 활용)
param_grid_lgb = {
    'n_estimators': [500],
    'learning_rate': [0.01],
    'num_leaves': [70],
    'max_depth': [20]
}
lgb_estimator = lgb.LGBMClassifier(random_state=42)
grid_search_lgb = GridSearchCV(
    estimator=lgb_estimator,
    param_grid=param_grid_lgb,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1
)
grid_search_lgb.fit(X_train, y_train)
best_lgb = grid_search_lgb.best_estimator_
print("LightGBM (GridSearchCV) 최적의 하이퍼파라미터:", grid_search_lgb.best_params_)

####################################
# Ensemble: 3개 모델의 예측 확률 평균 기반 보팅
prob_rf = best_rf.predict_proba(X_val)[:, 1]
prob_lgb_plain = best_lgb_plain.predict_proba(X_val)[:, 1]
prob_lgb_best = best_lgb.predict_proba(X_val)[:, 1]

# 세 모델의 예측 확률 평균 계산
ensemble_prob = (prob_rf + prob_lgb_plain + prob_lgb_best) / 3

# 평균 확률이 0.5 이상이면 1, 미만이면 0으로 최종 예측
ensemble_pred = (ensemble_prob >= 0.5).astype(int)

# 평가 지표 계산
ensemble_accuracy = accuracy_score(y_val, ensemble_pred)
ensemble_roc_auc = roc_auc_score(y_val, ensemble_prob)
ensemble_conf_mat = confusion_matrix(y_val, ensemble_pred)

print("Ensemble Accuracy: {:.4f}".format(ensemble_accuracy))
print("Ensemble ROC-AUC: {:.4f}".format(ensemble_roc_auc))
print("Ensemble Confusion Matrix:\n", ensemble_conf_mat)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
RandomForest 최적의 하이퍼파라미터: {'max_depth': 200, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 500}
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 45783, number of negative: 45782
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005948 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 160
[LightGBM] [Info] Number of data points in the train set: 91565, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500005 -> initscore=0.000022
[LightGBM] [Info] Start training from score 0.000022
LightGBM (기본 모델) 최적의 하이퍼파라미터: {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 300, 'num_leaves': 31}
Fitting 3 folds for ea

In [43]:
####################################
# 2. 테스트 데이터 예측
# 테스트 데이터 불러오기 (cp949 인코딩)
test_data = pd.read_csv('test2.csv', encoding='cp949')

# 테스트 데이터 전처리: 학습 시 사용했던 제외할 칼럼과 동일하게 제거
cols_to_drop_test = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부',
                '여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
                'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수',
                '동결 배아 사용 여부', '신선 배아 사용 여부']
X_test = test_data.drop(columns=cols_to_drop_test)

# 각 모델에 대한 예측 확률 계산
test_prob_rf = best_rf.predict_proba(X_test)[:, 1]
test_prob_lgb_plain = best_lgb_plain.predict_proba(X_test)[:, 1]
test_prob_lgb_best = best_lgb.predict_proba(X_test)[:, 1]

# 세 모델의 예측 확률 평균 계산 (Ensemble)
ensemble_test_prob = (test_prob_rf + test_prob_lgb_plain + test_prob_lgb_best) / 3
ensemble_test_pred = (ensemble_test_prob >= 0.5).astype(int)

# 예측 결과를 테스트 데이터에 추가
test_data['임신 성공 여부 예측'] = ensemble_test_pred
test_data['임신 성공 여부 예측_확률'] = ensemble_test_prob

# 결과 확인
print(test_data[['ID', '임신 성공 여부 예측', '임신 성공 여부 예측_확률']].head())

# 예측 결과를 CSV 파일로 저장
test_data.to_csv('test_ivf_ensemble_predictions_0225.csv', index=False, encoding='cp949')

           ID  임신 성공 여부 예측  임신 성공 여부 예측_확률
0  TEST_00000            0        0.017405
1  TEST_00001            0        0.017568
2  TEST_00002            0        0.382444
3  TEST_00003            0        0.255156
4  TEST_00004            1        0.644357
